In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import nest_asyncio

nest_asyncio.apply()

# Imports

In [ ]:
from typing import Literal, Optional

from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from langsmith import traceable
from pydantic import BaseModel
from typing_extensions import TypedDict

load_dotenv()

# Vanilla workflow

In [ ]:
class State(BaseModel):
    input: str
    type: Optional[
        Literal["write_article", "generate_table_of_contents", "review_article"]
    ] = None
    output: Optional[str] = None


class MessageType(BaseModel):
    type: Literal["write_article", "generate_table_of_contents", "review_article"]


model = ChatOpenAI(model="gpt-4.1-mini")


def classify_message(state: State) -> State:
    model_with_str_output = model.with_structured_output(MessageType)
    messages = [
        SystemMessage(
            content="You are a writer. You will classify the message into one of the following categories: 'write_article', 'generate_table_of_contents', 'review_article'."
        ),
        HumanMessage(content=f"Classify the message: {state.input}"),
    ]
    return model_with_str_output.invoke(messages).type


@traceable
def write_article(state: State) -> State:
    messages = [
        SystemMessage(
            content="You are a writer. You will write an article about the topic provided."
        ),
        HumanMessage(content=f"Write an article about {state.input}"),
    ]
    return model.invoke(messages).content


@traceable
def generate_table_of_contents(state: State) -> State:
    messages = [
        SystemMessage(
            content="You are a writer. You will generate a table of contents for an article about the topic provided."
        ),
        HumanMessage(
            content=f"Generate a table of contents for an article about {state.input}"
        ),
    ]
    return model.invoke(messages).content


@traceable
def review_article(state: State) -> State:
    messages = [
        SystemMessage(
            content="You are a writer. You will review the article for the topic provided."
        ),
        HumanMessage(content=f"Review the article for the topic {state.input}"),
    ]
    return model.invoke(messages).content


@traceable
def run_workflow(message: str) -> str:
    state = State(input=message)
    state.type = classify_message(state)
    if state.type == "write_article":
        return write_article(state)
    elif state.type == "generate_table_of_contents":
        return generate_table_of_contents(state)
    elif state.type == "review_article":
        return review_article(state)
    else:
        return "I'm sorry, I don't know how to handle that message."


run_workflow("Write an article about the meaning of life")

# LangGraph implementation

In [ ]:
class State(TypedDict):
    input: str
    type: Literal["write_article", "generate_table_of_contents", "review_article"]
    output: str


class MessageType(BaseModel):
    type: Literal["write_article", "generate_table_of_contents", "review_article"]


def classify_message(message: str) -> dict:
    model_with_str_output = model.with_structured_output(MessageType)
    messages = [
        SystemMessage(
            content="You are a writer. You will classify the message into one of the following categories: 'write_article', 'generate_table_of_contents', 'review_article'."
        ),
        HumanMessage(content=f"Classify the message: {message}"),
    ]
    return {"type": model_with_str_output.invoke(messages).type}


def route_message(state: State) -> State:
    if state["type"] == "write_article":
        return "generate_article_content"
    elif state["type"] == "generate_table_of_contents":
        return "generate_table_of_contents"
    elif state["type"] == "review_article":
        return "revise_article_content"
    else:
        raise ValueError(f"Invalid message type: {state['type']}")


def generate_table_of_contents(state: State) -> State:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, you will generate the table of contents for a short article."
        ),
        HumanMessage(
            content=f"Generate the table of contents of an article about {state['input']}"
        ),
    ]
    return {"output": model.invoke(messages).content}


def generate_article_content(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic and a table of contents, you will generate the content of the article."
        ),
        HumanMessage(
            content=f"Generate the content of an article about {state['input']}"
        ),
    ]
    return {"output": model.invoke(messages).content}


def revise_article_content(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, a table of contents and a content, you will revise the content of the article to make it less than 1000 characters."
        ),
        HumanMessage(
            content=f"Revise the content of the following article:\n\n{state['input']}"
        ),
    ]
    return {"output": model.invoke(messages).content}


workflow = StateGraph(State)

workflow.add_node("classify_message", classify_message)
workflow.add_conditional_edges(
    "classify_message",
    route_message,
    {
        "generate_article_content": "generate_article_content",
        "generate_table_of_contents": "generate_table_of_contents",
        "revise_article_content": "revise_article_content",
    },
)

workflow.add_node("generate_table_of_contents", generate_table_of_contents)
workflow.add_node("generate_article_content", generate_article_content)
workflow.add_node("revise_article_content", revise_article_content)

workflow.add_edge(START, "classify_message")
workflow.add_edge("generate_table_of_contents", END)
workflow.add_edge("generate_article_content", END)
workflow.add_edge("revise_article_content", END)

chain = workflow.compile()

display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:
ARTICLE = """
LangGraph is a library for building workflows. It helps you build workflows that are easy to understand and maintain.
"""

state = chain.invoke({"input": "I wrote this article please improve it:\n\n" + ARTICLE})

if "type" in state and state["type"] is not None:
    print("Type:")
    print(state["type"])
    print("\n--- --- ---\n")

if "output" in state:
    print("Output:")
    print(state["output"])
else:
    print("No output detected!")

## Exercise

Implement a LangGraph workflow that takes the content of a PDF file and depending on the type of document, it will process the document in a different way. Mock the processing for now.